## Install Packages

In [21]:
!conda config --add channels conda-forge
!conda install --yes geocoder

Solving environment: done

# All requested packages already installed.



## Import Libraries

In [22]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import bs4 as bs4
from bs4 import BeautifulSoup
import requests
import csv
import re
import urllib.request
import geocoder 

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')


Libraries imported.


## Import csv Lat Lng by Postal Code

In [23]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng = pd.DataFrame(df_latlng)
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Import data from previous assignment

In [24]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup1 = BeautifulSoup(source, 'lxml')
table = soup1.find('table', class_='wikitable sortable')
#table
A=[]
B=[]
C=[]

 
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
#print([A,B,C])

df=pd.DataFrame(A,columns=['Postalcode'])
df['Borough']=B
df['Neighborhood']=C
df = df[df.Borough != 'Not assigned']
df = df[df.Borough != '0']
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')

df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Retrieve and Loop Lat Lng

In [25]:
def get_latlng(postal_code):
    # apply None
    lat_lng_coord = None
    # loop for lat lng
    while(lat_lng_coord is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coord = g.latlng
    return lat_lng_coord
postal_codes = df['Postalcode']    
latlng = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [26]:
df_latlng = pd.DataFrame(latlng, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_latlng['Latitude']
df['Longitude'] = df_latlng['Longitude']

## Append the Lat Long to DataFrame

In [27]:
df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163
